<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorials/99_others/Eager_vs_PyTorch_speed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
tf.enable_eager_execution()

In [0]:
X = np.random.randn(10000, 32, 32, 3).astype(np.float32)
y = np.random.randint(0, 10, 10000).reshape(-1, 1).astype(np.int64)

In [0]:
data_set = tf.data.Dataset.from_tensor_slices((X, y))

In [0]:
data_set = data_set.shuffle(10000).batch(256)

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, activation="relu"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.Conv2D(32, 3, activation="relu"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.Conv2D(64, 3, activation="relu"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.Conv2D(128, 3, activation="relu"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.Conv2D(256, 3, activation="relu"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10),
])

In [32]:
model(tf.convert_to_tensor(X, dtype=tf.float32)[:2])

<tf.Tensor: id=484, shape=(2, 10), dtype=float32, numpy=
array([[ 0.05773694, -0.01897508, -0.05476988,  0.12189196, -0.00655457,
         0.0147602 ,  0.04510601,  0.05713235,  0.22796866,  0.02033146],
       [-0.01558806, -0.08503111, -0.07648266,  0.03848344, -0.02937521,
        -0.01581378,  0.00361122,  0.0868727 ,  0.12541714, -0.01783562]],
      dtype=float32)>

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  448       
_________________________________________________________________
batch_normalization (BatchNo multiple                  64        
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4640      
_________________________________________________________________
batch_normalization_1 (Batch multiple                  128       
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  18496     
_________________________________________________________________
batch_normalization_2 (Batch multiple                  256       
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  73856     
__________

In [0]:
def loss_fn(y, y_pre):
    return tf.keras.metrics.sparse_categorical_crossentropy(y, y_pre)

def acc_fn(y, y_pre):
    return tf.keras.metrics.sparse_categorical_accuracy(y, y_pre)

optimizer = tf.train.AdamOptimizer()

In [0]:
def train(x, y):
    with tf.device("/gpu:0"):
        with tf.GradientTape() as g:
            y_pre = model(x)
            loss = tf.reduce_sum(loss_fn(y, y_pre))
        grads = g.gradient(loss, model.variables)
        acc = tf.reduce_mean(acc_fn(y, y_pre))
        optimizer.apply_gradients(zip(grads, model.variables))
    return loss, acc

train_graph = tf.contrib.eager.defun(train)

In [40]:
%%time
for _ in range(10):
    
    running_loss = 0
    running_acc = 0
    for i, (x_, y_) in enumerate(data_set):
#         loss, acc, = train(x_, y_)
        loss, acc, = train_graph(x_, y_)
        running_loss += loss
        running_acc += acc
    
    print("***********************")
    print("loss", running_loss.numpy() / (i+1) )
    print("acc", running_acc.numpy() / (i+1) )    

***********************
loss 2.3023305892944337
acc 0.1037109375
***********************
loss 2.3023033142089844
acc 0.10517578125
***********************
loss 2.302598571777344
acc 0.10224609375
***********************
loss 2.3023960113525392
acc 0.10517578125
***********************
loss 2.3023252487182617
acc 0.10517578125
***********************
loss 2.3024168014526367
acc 0.1037109375
***********************
loss 2.302304267883301
acc 0.10517578125
***********************
loss 2.3022958755493166
acc 0.10517578125
***********************
loss 2.3024080276489256
acc 0.10224609375
***********************
loss 2.3023279190063475
acc 0.10224609375
CPU times: user 1min 22s, sys: 28.7 s, total: 1min 51s
Wall time: 1min 54s


In [76]:
!pip install torch

    100% |████████████████████████████████| 591.8MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x61abc000 @  0x7f40d3dea2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

In [0]:
X_torch = X.transpose(0, 3, 2, 1)
y_torch = y.reshape(-1)

In [0]:
model_conv = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(16),
    nn.Conv2d(16, 32, 3),
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(32),
    nn.Conv2d(32, 64, 3),
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(64),
    nn.Conv2d(64, 128, 3),
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(128),
    nn.Conv2d(128, 256, 3),
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(256),
    nn.Dropout2d(0.5)
)

In [10]:
model_conv(torch.from_numpy(X_torch)[:2])

tensor([[[[ 1.5284, -0.9712, -0.9712,  ..., -0.8616, -0.9712, -0.9712],
          [-0.4848, -0.9634, -0.9712,  ...,  3.0634, -0.9712, -0.3867],
          [-0.9712, -0.9712,  0.2896,  ...,  1.3941,  0.2910,  2.0000],
          ...,
          [-0.1392, -0.7536, -0.1402,  ..., -0.8794,  2.5814, -0.9712],
          [ 2.1451, -0.9712, -0.4530,  ..., -0.9712, -0.9712, -0.9712],
          [-0.9712, -0.9712, -0.9712,  ..., -0.9712, -0.9712, -0.9712]],

         [[ 0.0000, -0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0000],
          ...,
          [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000,  ...,  0.0000, -0.0000, -0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -

In [0]:
class Model_torch(nn.Module):
    def __init__(self):
        super(Model_torch, self).__init__()
        self.conv = model_conv
        self.linear = nn.Sequential(
            nn.Linear(256*22*22, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(1024, 10),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 256*22*22).contiguous()
        x = self.linear(x)
        return x

In [0]:
model_torch = Model_torch().to("cuda")
model_torch

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_torch.parameters())

In [0]:
data_set_torch = torch.utils.data.TensorDataset(torch.from_numpy(X_torch), 
                                                torch.from_numpy(y_torch))

In [0]:
dataset_torch = torch.utils.data.DataLoader(data_set_torch, batch_size=256)


In [0]:
def train_torch(x, y):
    
    optimizer.zero_grad() 
    y_pre = model_torch(x)
    loss = criterion(y_pre, y)
    loss.backward()
    optimizer.step()
    _, preds = torch.max(y_pre, 1)
    acc = torch.sum(preds == y) / y.size(0)
    return loss.item(), acc.item()


In [25]:
%%time
for _ in range(10):
    
    running_loss = 0
    running_acc = 0
    for i, (x_, y_) in enumerate(dataset_torch):
        loss, acc, = train_torch(x_.to("cuda"), y_.to("cuda"))
        running_loss += loss
        running_acc += acc
    
    print("***********************")
    print("loss", running_loss / (i+1) )
    print("acc", running_acc / (i+1) )    

***********************
loss 0.002378651115577668
acc 0.975
***********************
loss 0.0015758836641907692
acc 1.0
***********************
loss 0.0023638030514121057
acc 0.9
***********************
loss 0.002076438756193966
acc 0.975
***********************
loss 0.0016074220882728695
acc 0.975
***********************
loss 0.0013330027111805975
acc 0.95
***********************
loss 0.001690656109713018
acc 0.9
***********************
loss 0.0021041302708908914
acc 0.95
***********************
loss 0.003921843133866787
acc 0.775
***********************
loss 0.004420859855599701
acc 0.85
CPU times: user 1min 27s, sys: 52.1 s, total: 2min 19s
Wall time: 2min 20s
